"4.Use logistic regression to predict the 90-day repeat of users acquired in November 2018.
Hint: Use Source, Slot of Booking, Gap between date of Booking and date of service requested as predictor variables"

In [374]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import f1_score

I took my train-test data from December to August to predict the 90 day repeat of users acquired in November 2018 !

In [375]:
import pandas as pd 
from collections import OrderedDict
import pickle
df = pd.read_pickle("features.pickle")
df.head()

,Transaction_ID,Profile ID,Date_of_Booking,Date_of_Service_Requested,Source,Slot of Booking (Hour of the Day),year,month,day,days_since_last_event,DateofServiceRequested,diff_minutes,days_sincelast_event,DateofBooking,latest_cart,labels,thirty,ninty
0,BBCHH,1,2018-05-20,2018-05-21,D,14,2018,5,20,NaT,2018-05-21,1 days,0.0,2018-05-20,"[2018-05-20 00:00:00, 2018-09-23 00:00:00]",126.0,0,0
1,CHWFD,1,2018-09-23,2018-09-23,C,8,2018,9,23,126 days,2018-09-23,0 days,126.0,2018-09-23,"[2018-05-20 00:00:00, 2018-09-23 00:00:00]",0.0,0,0
2,DYDMF,2,2018-11-10,2018-11-13,C,11,2018,11,10,NaT,2018-11-13,3 days,0.0,2018-11-10,[2018-11-10 00:00:00],0.0,0,0
3,EZYSA,3,2018-04-12,2018-04-16,B,8,2018,4,12,NaT,2018-04-16,4 days,0.0,2018-04-12,"[2018-04-12 00:00:00, 2018-08-05 00:00:00]",115.0,0,0
4,HWAKX,3,2018-08-05,2018-08-06,B,11,2018,8,5,115 days,2018-08-06,1 days,115.0,2018-08-05,"[2018-04-12 00:00:00, 2018-08-05 00:00:00]",0.0,0,0


"4.Use logistic regression to predict the 90-day repeat of users acquired in November 2018.
Hint: Use Source, Slot of Booking, Gap between date of Booking and date of service requested as predictor variables"

In [376]:
df.rename(columns={'diff_minutes':'diff'},inplace=True)
df.rename(columns={'days_Since_next_order':'days_to_next_order'},inplace=True)

df.head()

,Transaction_ID,Profile ID,Date_of_Booking,Date_of_Service_Requested,Source,Slot of Booking (Hour of the Day),year,month,day,days_since_last_event,DateofServiceRequested,diff,days_sincelast_event,DateofBooking,latest_cart,labels,thirty,ninty
0,BBCHH,1,2018-05-20,2018-05-21,D,14,2018,5,20,NaT,2018-05-21,1 days,0.0,2018-05-20,"[2018-05-20 00:00:00, 2018-09-23 00:00:00]",126.0,0,0
1,CHWFD,1,2018-09-23,2018-09-23,C,8,2018,9,23,126 days,2018-09-23,0 days,126.0,2018-09-23,"[2018-05-20 00:00:00, 2018-09-23 00:00:00]",0.0,0,0
2,DYDMF,2,2018-11-10,2018-11-13,C,11,2018,11,10,NaT,2018-11-13,3 days,0.0,2018-11-10,[2018-11-10 00:00:00],0.0,0,0
3,EZYSA,3,2018-04-12,2018-04-16,B,8,2018,4,12,NaT,2018-04-16,4 days,0.0,2018-04-12,"[2018-04-12 00:00:00, 2018-08-05 00:00:00]",115.0,0,0
4,HWAKX,3,2018-08-05,2018-08-06,B,11,2018,8,5,115 days,2018-08-06,1 days,115.0,2018-08-05,"[2018-04-12 00:00:00, 2018-08-05 00:00:00]",0.0,0,0


In [377]:
df_feature=df.loc[:,['Source','Slot of Booking (Hour of the Day)','diff','ninty','month']]

In [378]:
df_feature.head(20)

,Source,Slot of Booking (Hour of the Day),diff,ninty,month
0,D,14,1 days,0,5
1,C,8,0 days,0,9
2,C,11,3 days,0,11
3,B,8,4 days,0,4
4,B,11,1 days,0,8
5,A,13,3 days,0,3
6,C,14,1 days,0,1
7,B,13,3 days,0,7
8,D,16,5 days,0,12
9,C,10,0 days,1,11


In [379]:
df_feature = pd.concat([df_feature, pd.get_dummies(df_feature['Source'])], axis=1)


In [380]:

df_feature.rename(columns={'A':'Source_A','B':'Source_B','C':'Source_C','D':'Source_D'},inplace=True)
df_feature.head()


,Source,Slot of Booking (Hour of the Day),diff,ninty,month,Source_A,Source_B,Source_C,Source_D
0,D,14,1 days,0,5,0,0,0,1
1,C,8,0 days,0,9,0,0,1,0
2,C,11,3 days,0,11,0,0,1,0
3,B,8,4 days,0,4,0,1,0,0
4,B,11,1 days,0,8,0,1,0,0


In [381]:
del df_feature['Source']

In [382]:
def func(x):
    x=x.days
    x=str(x)
    return x[0]
df_feature['diff_row']=df['diff'].map(func)

In [383]:
df_feature.head()

,Slot of Booking (Hour of the Day),diff,ninty,month,Source_A,Source_B,Source_C,Source_D,diff_row
0,14,1 days,0,5,0,0,0,1,1
1,8,0 days,0,9,0,0,1,0,0
2,11,3 days,0,11,0,0,1,0,3
3,8,4 days,0,4,0,1,0,0,4
4,11,1 days,0,8,0,1,0,0,1


In [384]:
del df_feature['diff']


In [385]:
df_feature.tail()
df_feature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30940 entries, 0 to 30939
Data columns (total 8 columns):
Slot of Booking (Hour of the Day)    30940 non-null int64
ninty                                30940 non-null int64
month                                30940 non-null int64
Source_A                             30940 non-null uint8
Source_B                             30940 non-null uint8
Source_C                             30940 non-null uint8
Source_D                             30940 non-null uint8
diff_row                             30940 non-null object
dtypes: int64(3), object(1), uint8(4)
memory usage: 1.3+ MB


In [386]:
df_feature.set_index(['month'],inplace=True)

In [387]:
df_feature

,Slot of Booking (Hour of the Day),ninty,Source_A,Source_B,Source_C,Source_D,diff_row
month,,,,,,,
5,14,0,0,0,0,1,1
9,8,0,0,0,1,0,0
11,11,0,0,0,1,0,3
4,8,0,0,1,0,0,4
8,11,0,0,1,0,0,1
3,13,0,1,0,0,0,3
1,14,0,0,0,1,0,1
7,13,0,0,1,0,0,3
12,16,0,0,0,0,1,5


In [388]:
df_model=df_feature.loc[[12,1,2,3,4,5,6,7,8],:]


In [389]:
df_model

,Slot of Booking (Hour of the Day),ninty,Source_A,Source_B,Source_C,Source_D,diff_row
month,,,,,,,
12,16,0,0,0,0,1,5
12,14,0,0,0,1,0,4
12,16,0,1,0,0,0,4
12,11,0,0,1,0,0,4
12,14,1,0,1,0,0,0
12,14,0,0,0,1,0,4
12,14,1,0,0,1,0,5
12,19,0,0,0,1,0,5
12,19,0,0,0,0,1,2


In [390]:
df_nov=df_feature.loc[11,:]

In [391]:
df_nov

,Slot of Booking (Hour of the Day),ninty,Source_A,Source_B,Source_C,Source_D,diff_row
month,,,,,,,
11,11,0,0,0,1,0,3
11,10,1,0,0,1,0,0
11,14,0,0,0,1,0,0
11,13,0,0,0,1,0,1
11,8,0,0,1,0,0,0
11,13,0,0,0,1,0,5
11,14,0,1,0,0,0,0
11,14,0,0,0,1,0,1
11,14,0,0,0,1,0,1


In [392]:
df_model.head()
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23175 entries, 12 to 8
Data columns (total 7 columns):
Slot of Booking (Hour of the Day)    23175 non-null int64
ninty                                23175 non-null int64
Source_A                             23175 non-null uint8
Source_B                             23175 non-null uint8
Source_C                             23175 non-null uint8
Source_D                             23175 non-null uint8
diff_row                             23175 non-null object
dtypes: int64(2), object(1), uint8(4)
memory usage: 814.7+ KB


In [393]:
import numpy as np
from sklearn.model_selection import train_test_split

In [394]:
train_df, test_df = train_test_split(df_feature, test_size=0.2)

# Form np arrays of labels and features.
train_y = np.array(train_df.pop('ninty'))
test_y = np.array(test_df.pop('ninty'))

train_x= np.array(train_df)
test_x = np.array(test_df)

# Normalize the input features using the sklearn StandardScaler.
# This will set the mean to 0 and standard deviation to 1.
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [395]:
## print out the shapes of  resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(24752, 6) 
Test set: 		(6188, 6)


In [396]:
train_x

array([[ 0.30451102, -0.44673652, -0.57050791,  1.4040757 , -0.57971392,
        -0.29591235],
       [ 1.26129901,  2.2384559 , -0.57050791, -0.71221231, -0.57971392,
        -0.29591235],
       [ 0.30451102, -0.44673652, -0.57050791,  1.4040757 , -0.57971392,
         0.28919889],
       ...,
       [-0.01441831, -0.44673652,  1.75282407, -0.71221231, -0.57971392,
        -0.8810236 ],
       [ 0.30451102, -0.44673652, -0.57050791,  1.4040757 , -0.57971392,
        -1.46613485],
       [-0.01441831, -0.44673652, -0.57050791,  1.4040757 , -0.57971392,
        -0.29591235]])

In [397]:
bincount=np.bincount(train_y)
bincount

array([16212,  8540], dtype=int64)

In [398]:
zero, pos = np.bincount(train_y)
print(zero)


16212


In [399]:
total = zero + pos
print('{} positive samples out of {} training samples ({:.2f}% of total)'.format(
    zero, total, 100 * pos / total))

16212 positive samples out of 24752 training samples (34.50% of total)


Due to uneuqal proprtion of labels. I use class_weight = balanced to account for less no of positive sample in the dataset.

In [400]:
#lr = LogisticRegression(C=1000) 
lr_balanced = LogisticRegression(class_weight='balanced')
#lr_10x = LogisticRegression(class_weight={1 :1.9, 0 : 1},C=100)
    
#lr.fit(train_x, train_y)
#f1_score = f1_score(lr.predict(test_x), test_y)

lr_balanced.fit(train_x, train_y)
f1_score_balanced = f1_score(lr_balanced.predict(test_x), test_y)

#lr_10x.fit(train_x, train_y)
#f1_score_lr_10x = f1_score(lr_10x.predict(test_x), test_y)  




C:\Users\akash\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [401]:

#print(f1_score)
print(f1_score_balanced)
#print(f1_score_lr_10x)





0.3884863123993559


In [402]:
nov_y = np.array(df_nov.pop('ninty'))
nov_x= np.array(df_nov)
print(nov_x)


# Normalize the input features using the sklearn StandardScaler.
# This will set the mean to 0 and standard deviation to 1.
scaler = StandardScaler()
nov_x = scaler.fit_transform(nov_x)

[[11 0 0 1 0 '3']
 [10 0 0 1 0 '0']
 [14 0 0 1 0 '0']
 ...
 [13 0 0 0 1 '5']
 [11 0 0 1 0 '4']
 [14 0 0 1 0 '2']]


In [403]:
## print out the shapes of  resultant feature data
print("\t\t\tFeature Shapes:")
print("feature set: \t\t{}".format(nov_x.shape),
      "\nlabel set: \t\t{}".format(nov_y.shape))

			Feature Shapes:
feature set: 		(2577, 6) 
label set: 		(2577,)


In [404]:
x=lr_balanced.predict(nov_x)
bincount=np.bincount(x)
bincount

array([1395, 1182], dtype=int64)

Out of total 2577 users in NOV 1182 will be order within next 90 days.

Thank You !`